<center> <span style="color:indigo">Machine Learning e Inferencia Bayesiana</span> </center> 

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/5/5e/Logo-cucea.png" alt="Drawing" style="width: 600px;"/>
</center>
    
<center> <span style="color:DarkBlue">  Tema 4: Probabilidad, Naive Bayes, datos ficticios de emails </span>  </center>
<center> <span style="color:Blue"> M. en C. Iván A. Toledano Juárez </span>  </center>

## Clasificación de Correos SPAM usando Machine Learning

Este ejercicio forma parte de las **notas del curso de *Machine Learning e Inferencia Bayesiana***. Trabajaremos un caso clásico de **clasificación binaria**: la detección de mensajes **SPAM** en una bandeja de entrada ficticia.

Utilizaremos un conjunto de datos *simulado* que contiene ejemplos de correos electrónicos, donde cada uno está etiquetado como **SPAM** o **NO SPAM**. Un ejemplo de mensaje típico es:

> *"Congratulations, you won free gift!"*

Este tipo de mensaje contiene palabras clave como `"congratulations"`, `"won"`, `"free"` y `"gift"`, frecuentemente asociadas al correo no deseado.

---

En este notebook realizaremos el análisis en dos enfoques complementarios:

- **Implementación manual del modelo Naive Bayes**, paso a paso, para comprender a profundidad los fundamentos teóricos y probabilísticos.
- **Uso de Scikit-learn** para validar el modelo.


In [2]:
# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB, BernoulliNB, CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix

## Carga de datos

In [6]:
df_spam = pd.read_csv('../data/spam/spam_ficticio.csv')
df_spam.head(10)

,congratulations,you,won,free,gift,attached,sincerely,thanks,spam
0,1,1,0,1,0,0,1,0,1
1,1,1,1,1,0,0,0,1,1
2,1,1,1,1,1,1,1,0,1
3,1,0,1,1,1,0,0,1,1
4,1,0,1,1,1,0,0,1,1
5,0,0,1,1,1,0,1,0,1
6,0,1,0,1,0,1,0,1,0
7,0,1,0,0,1,1,0,1,0
8,0,0,0,0,0,1,1,0,0
9,1,0,1,0,0,0,1,0,0


### Implementación manual del clasificador Naive Bayes

A continuación, se presenta una función personalizada (`nb_casero`) que implementa desde cero el algoritmo **Naive Bayes multivariado binario**. Esta función sirve como ejemplo claro de cómo se aplica el **teorema de Bayes** para calcular las probabilidades posteriores para cada clase, bajo el supuesto de independencia condicional entre las variables predictoras.

La función trabaja de la siguiente manera:

- Se calcula la **tabla de probabilidades condicionales** $P(x_i = 1 \mid y)$ para cada variable binaria `x_i` y clase `y`, utilizando los datos de entrenamiento.
- Se aplican las fórmulas:
  
  \begin{equation}
  P(y=1 \mid \mathbf{x}) \propto P(y=1) \cdot \prod_i P(x_i \mid y=1)
  \end{equation}
  \begin{equation}
  P(y=0 \mid \mathbf{x}) \propto P(y=0) \cdot \prod_i P(x_i \mid y=0)
  \end{equation}
  
- Para cada observación en el conjunto de prueba, se calcula la probabilidad posterior para cada clase (sin normalizar) y se asigna la clase con mayor probabilidad.

Además, para cada observación se imprime en consola el valor de las probabilidades condicionales calculadas (`p_0`, `p_1`) y la predicción final, permitiendo una comprensión detallada de cada paso.

Esta implementación resalta de forma didáctica los siguientes conceptos clave:

- Cálculo de la verosimilitud (*likelihood*) para variables binarias.
- Aplicación del teorema de Bayes en clasificación.
- Comparación entre las probabilidades para tomar la decisión final.

In [11]:
# Función custom para hacer la clasificación en base al teorema de Bayes
def nb_casero(df_train,df_test,features,target):
    
    # tabla de probabilidad(likelihood)
    df_prob = df_train.groupby(target).mean()
    
    p_0_list = []
    p_1_list = []
    pred_list=[] # lista de predicciones

    # Probabilidades totales del target
    p_target_0 = df_train.groupby(target).size()[0]/df_train.shape[0]
    p_target_1 = df_train.groupby(target).size()[1]/df_train.shape[0]

    # iteramos sobre todos los registros(filas) utilizando método iterrows()
    for index, row in df_test.iterrows():
        
        p_0 = p_target_0 # inicializamos la probabilidad condicional total
        p_1 = p_target_1 # inicializamos la probabilidad condicional total

        #iteracion sobre las variables
        for var in features:
            if row[var]==1: # caso var=1
                p_0 *= df_prob[var].loc[0] # se multiplica por su respectiva entrada en la tabla de prob.
                p_1 *= df_prob[var].loc[1]
            elif row[var]==0: # caso var=0
                p_0 *= 1.0-df_prob[var].loc[0] # se multiplica por su respectivo complemento en la tabla de prob.
                p_1 *= 1.0-df_prob[var].loc[1]
        
        p_0_list.append(p_0)
        p_1_list.append(p_1)
        pred = np.where(p_1>p_0,1,0) # Se comparan las dos prob condicionales, se elige la etiqueta de la mayor
        pred_list.append(int(pred)) # Se agrega a la lista de predicciones
        print('p_0=',round(p_0,5),'p_1=',round(p_1,5),'pred=',int(pred))
        
    return p_0_list, p_1_list, pred_list
    

### Implementación de la función casera

In [12]:
# Dataframe de entrenamiento
df_train = pd.read_csv('../data/spam/spam_ficticio.csv')

# Lista de variables de entrada y objetivo
target = 'spam'
features = df_train.columns.drop(target)

# Dataframe de prueba
df_test = pd.DataFrame(data={},
                       columns=features)
# Añadimos un renglón al set de validacion
df_test.loc[len(df_test.index)] = [1,1,1,1,1,0,0,0] # codificacion de mensaje "congratulations, you won free gift"

p_0_list, p_1_list, pred_list = nb_casero(df_train=df_train,
                                          df_test=df_test,
                                          features=features,
                                          target=target)

p_0= 5e-05 p_1= 0.02894 pred= 1
